In [1]:

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.colors import LinearSegmentedColormap, Normalize

import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar
from core.s3 import S3AssetManager


In [2]:

notebook_name = "central_ganadera_microbiologia_overview"
s3 = S3AssetManager(notebook_name=notebook_name)
CORP_PALETTE =  [ "#1c8074","#1a494c", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]

In [3]:
df_raw = s3.read_excel("raw/central_ganadera/CG Data.xlsx", sheet_name="Sheet1")

In [4]:
#A91
df_raw.head()

,=RowNo(TOTAL),Ensayo,Nombre ensayo,Fecha de muestreo.autoCalendar.Year,Fecha de muestreo.autoCalendar.Month,Fecha de muestreo.autoCalendar.Date,Lote,Tipo de planta,Código de empresa,Código de planta,...,Límite,Cumplimiento,[Análisis id],orden_etapa,Archivo fuente,Ciudad proveedor,Departamento proveedor,Manipulador,Municipio,Operación unitaria
0,1,-,-,2023,Abr,2023-04-04,-,Planta de beneficio,A91,A91-A,...,0,100,0a0a63d2a2ffb6a40da076b5dd7fea5b,-,RG_L_04_Informe_Resultado_Laboratorio_2023.xlsx,-,-,-,-,Producto terminado
1,2,-,-,2025,May,2025-05-20,600 451 10,Planta de beneficio,A91,A91-A,...,3,100,0a0b79cb0f1826206e010be82f8a16e7,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,Anori,Producto terminado
2,3,-,-,2020,Nov,2020-11-20,-,Planta de beneficio,A91,A91-A,...,5,100,0a0bc472af5103004e9ed7925d919948,-,RG_L_04_Informe_Resultado_Laboratorio_2020.xlsx,-,-,-,-,Sierras
3,4,-,-,2021,Feb,2021-02-05,-,Planta de beneficio,A91,A91-A,...,10,100,0a0d2d233363f9ab7f5d95e447b8e68f,-,RG_L_04_Informe_Resultado_Laboratorio_2021.xlsx,-,-,-,-,Producto terminado
4,5,-,-,2022,Jun,2022-06-06,-,Planta de beneficio,A91,A91-A,...,0,100,0a1aaf1289e4e0188c4431daff410651,-,RG_L_04_Informe_Resultado_Laboratorio_2022.xlsx,-,-,-,-,Producto terminado


In [5]:
df_raw["Fecha de muestreo.autoCalendar.Date"].min(), df_raw["Fecha de muestreo.autoCalendar.Date"].max()

(Timestamp('2018-01-26 00:00:00'), Timestamp('2025-08-26 00:00:00'))

In [6]:
df_raw["Código de empresa"].value_counts()

A91    92562
Name: Código de empresa, dtype: int64

In [7]:
df_raw["Línea de producción"].value_counts(normalize=True)

Línea bovinos      0.451946
Línea porcinos     0.323826
Sbe                0.140976
Sin identificar    0.081653
Tanques de agua    0.001599
Name: Línea de producción, dtype: float64

In [8]:
df_raw["Categoría"].value_counts()

No aplica          57931
POES               22392
Manipulador         5296
Ambiente            3439
Empaques            2178
Agua                1290
Aire comprimido       36
Name: Categoría, dtype: int64

In [9]:
df = df_raw[df_raw["Línea de producción"] == "Línea porcinos"].copy()
df["deteccion"] = np.where(df["Resultado"]>0, 1, 0)
df["Fecha de muestreo.autoCalendar.Date"] = pd.to_datetime(df["Fecha de muestreo.autoCalendar.Date"])
df["year_month"] = pd.to_datetime(df["Fecha de muestreo.autoCalendar.Date"]).dt.strftime("%Y-%m")


In [10]:
df.shape

(29974, 41)

In [11]:
df["Inductor"].value_counts(normalize=True)

Producto terminado    0.63058
Control de proceso    0.36942
Name: Inductor, dtype: float64

In [12]:
df["Etapa de proceso"].value_counts(normalize=True)

Producto terminado     0.630580
Planta porcinos        0.160939
Producción porcinos    0.135884
Frío porcinos          0.072596
Name: Etapa de proceso, dtype: float64

In [13]:
df["Muestra"].value_counts(normalize=True)

Canal Porcina                         0.423500
Vr porcina                            0.088777
Buche Porcino                         0.068159
Madeja Porcina                        0.036598
Guarda descenso canales porcino       0.013679
                                        ...   
Manipulador 35 Calidad Porcinos T2    0.000067
Manipulador 33 Linea Porcinos T2      0.000067
Manipulador 41 Frío Porcinos T1       0.000067
Canal retenida porcinos               0.000033
Ambiente área de canastas             0.000033
Name: Muestra, Length: 97, dtype: float64

In [14]:
df["Proveedor"].value_counts(normalize=True)

-                           0.962868
Hacienda San Jose           0.001101
La Sultana                  0.001101
La Carolina                 0.000901
El Porvenir Velsagro Sas    0.000801
                              ...   
La Esperanza                0.000033
Villa Rubia Yarumos         0.000033
El Recreo                   0.000033
El Rio                      0.000033
Santa Ines                  0.000033
Name: Proveedor, Length: 255, dtype: float64

In [15]:
df["Granja"].value_counts(normalize=True)

-    1.0
Name: Granja, dtype: float64

In [16]:
df["Microorganismo"].value_counts(normalize=True)

Rec. Coliformes totales    0.430406
Rec. E. coli               0.418129
Det. Salmonella spp.       0.111563
Det. E. coli O157:H7       0.026356
Antiobióticos              0.005838
Det. Trichinella           0.005004
Met. pesados               0.002669
Rec. Aerobios mesófilos    0.000033
Name: Microorganismo, dtype: float64

In [17]:
df["orden_etapa"].value_counts(normalize=True)

-    0.999867
0    0.000133
Name: orden_etapa, dtype: float64

In [18]:
df["Manipulador"].value_counts(normalize=True)

-                       0.911924
Sebastian Mejia         0.002135
Juan Carlos Gonzalez    0.001735
Jorge Agudelo           0.001334
Pedro Castaneda         0.001334
                          ...   
Jose Caly Pena          0.000067
Johany Rojas            0.000067
Miguel Acosta           0.000067
Luis Ramirez            0.000067
Leonardo Durango        0.000067
Name: Manipulador, Length: 507, dtype: float64

In [19]:
df["Operación unitaria"].value_counts(normalize=True)

Producto terminado       0.630580
Sin identificar          0.087876
Guardas                  0.040769
Guantes                  0.035164
Termómetros              0.018816
Esparrancador            0.017882
Ganchos                  0.015280
Andamios                 0.015146
Punzones                 0.012678
Mesa                     0.009675
Carros                   0.009141
Astilladora              0.009141
Canastas                 0.008974
Lavado productos         0.008941
Escabiladero             0.008941
Repeladora               0.008941
Lanceta                  0.008941
Pinzas                   0.008941
Cascos y overoles        0.008541
Cuchillos                0.006205
Cavas                    0.005471
Manos                    0.005271
Lavado vísceras          0.004471
Lavado canales           0.004471
Cuchilla                 0.003670
Toombler                 0.003670
Guía toma de muestras    0.002135
Contingencia             0.000267
Name: Operación unitaria, dtype: float64

In [20]:
df_ =df[df["Lote"] !=  '-']
df_dupl = df_[df_.duplicated('Lote', keep=False)].sort_values('Lote', ascending=False)
df_dupl["Operación unitaria"].value_counts(dropna=False)

Producto terminado    1585
Name: Operación unitaria, dtype: int64

In [21]:
df_filtrado_ = df_dupl.groupby("Lote").filter(lambda g: (g["Resultado"] > 0).all())
df_filtrado_

,=RowNo(TOTAL),Ensayo,Nombre ensayo,Fecha de muestreo.autoCalendar.Year,Fecha de muestreo.autoCalendar.Month,Fecha de muestreo.autoCalendar.Date,Lote,Tipo de planta,Código de empresa,Código de planta,...,[Análisis id],orden_etapa,Archivo fuente,Ciudad proveedor,Departamento proveedor,Manipulador,Municipio,Operación unitaria,deteccion,year_month
58737,58738,-,-,2025,May,2025-05-20,bache 3 buche 30,Planta de beneficio,A91,A91-A,...,95020a061575f1b46b74d510388203f6,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-05
43356,43357,-,-,2025,May,2025-05-20,bache 3 buche 30,Planta de beneficio,A91,A91-A,...,2a41e1a8e117819771b28b31b1486bf5,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-05
61017,61018,-,-,2025,Feb,2025-02-10,9971 60,Planta de beneficio,A91,A91-A,...,a29b83da68d9f11ad4efaff6cdf92e0e,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
61329,61330,-,-,2025,Feb,2025-02-10,9971 60,Planta de beneficio,A91,A91-A,...,a589ad11fadf4d3091ae0fdc46c9644f,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
50842,50843,-,-,2025,Feb,2025-02-10,9971 29,Planta de beneficio,A91,A91-A,...,43dc532fafe8d3c727350a5c7c4c6971,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59446,59447,-,-,2025,Feb,2025-02-12,0078 1,Planta de beneficio,A91,A91-A,...,1702950bae0086ea13b44db2924be02f,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
45094,45095,-,-,2025,Feb,2025-02-12,0062 23,Planta de beneficio,A91,A91-A,...,4c41d635575e51735eebecf64c30361c,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
66827,66828,-,-,2025,Feb,2025-02-12,0062 23,Planta de beneficio,A91,A91-A,...,d70de6fc123afffb94fe403b4bfb95ac,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02
63261,63262,-,-,2025,Feb,2025-02-11,0029 30,Planta de beneficio,A91,A91-A,...,b957b47a1d7cd2bfaff35a83eea99997,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-02


In [22]:
df_filtrado_["Microorganismo"].value_counts()

Rec. Coliformes totales    69
Rec. E. coli               69
Det. Salmonella spp.        3
Name: Microorganismo, dtype: int64

In [23]:
df[df["Fecha de muestreo.autoCalendar.Date"]=='2025-01-25']

,=RowNo(TOTAL),Ensayo,Nombre ensayo,Fecha de muestreo.autoCalendar.Year,Fecha de muestreo.autoCalendar.Month,Fecha de muestreo.autoCalendar.Date,Lote,Tipo de planta,Código de empresa,Código de planta,...,[Análisis id],orden_etapa,Archivo fuente,Ciudad proveedor,Departamento proveedor,Manipulador,Municipio,Operación unitaria,deteccion,year_month
46576,46577,-,-,2025,Ene,2025-01-25,9249 12,Planta de beneficio,A91,A91-A,...,6c8c402d8cd315637f412ed07c1e156b,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-01
47667,47668,-,-,2025,Ene,2025-01-25,9262 19,Planta de beneficio,A91,A91-A,...,7f4fa82a748d8536a14973a68b6598f5,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-01
50970,50971,-,-,2025,Ene,2025-01-25,9249 12,Planta de beneficio,A91,A91-A,...,46b97096545ec844664d33c97bb73da9,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,0,2025-01
52903,52904,-,-,2025,Ene,2025-01-25,9233 26,Planta de beneficio,A91,A91-A,...,86b0553ce226d0fa8a495dde646796e7,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-01
58589,58590,-,-,2025,Ene,2025-01-25,9262 19,Planta de beneficio,A91,A91-A,...,82440b4e559058c40fded75e9cd1c468,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-01
68776,68777,-,-,2025,Ene,2025-01-25,9233 26,Planta de beneficio,A91,A91-A,...,e76bcd9206361b70b436c41412f48aa7,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,1,2025-01


In [24]:
df["Etapa de proceso"].value_counts(dropna=False)

Producto terminado     18901
Planta porcinos         4824
Producción porcinos     4073
Frío porcinos           2176
Name: Etapa de proceso, dtype: int64

In [25]:

todas_etapas = set(df["Etapa de proceso"].unique())

# Agrupar por fecha y filtrar aquellas que contengan todas las etapas
fechas_completas = (
    df.groupby("Fecha de muestreo.autoCalendar.Date")["Etapa de proceso"]
      .apply(lambda etapas: set(etapas) == todas_etapas)
)
fechas_validas = fechas_completas[fechas_completas].index.tolist()
df_comp = df[df["Fecha de muestreo.autoCalendar.Date"].isin(fechas_validas)]

In [26]:
df_comp

,=RowNo(TOTAL),Ensayo,Nombre ensayo,Fecha de muestreo.autoCalendar.Year,Fecha de muestreo.autoCalendar.Month,Fecha de muestreo.autoCalendar.Date,Lote,Tipo de planta,Código de empresa,Código de planta,...,[Análisis id],orden_etapa,Archivo fuente,Ciudad proveedor,Departamento proveedor,Manipulador,Municipio,Operación unitaria,deteccion,year_month
41836,41837,-,-,2022,May,2022-05-16,-,Planta de beneficio,A91,A91-A,...,0a1d1184514d487a8c1e0d52efab0330,-,RG_L_04_Informe_Resultado_Laboratorio_2022.xlsx,-,-,-,-,Lanceta,1,2022-05
41837,41838,-,-,2025,Abr,2025-04-23,bache 3 buche 1,Planta de beneficio,A91,A91-A,...,0a1e9fae84596e8b3d1f480f78a48179,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,-,Producto terminado,0,2025-04
41843,41844,-,-,2025,Jul,2025-07-31,8160 43,Planta de beneficio,A91,A91-A,...,0a6a6f379f459c920ca91f4be572f3eb,-,RG L 04 Informe Resultado Laboratorio 2025.xlsx,-,-,-,Medellin,Producto terminado,0,2025-07
41844,41845,-,-,2024,Abr,2024-04-19,-,Planta de beneficio,A91,A91-A,...,0a6ae5adbc188715a320ed26750675ec,-,RG_L_04_Informe_Resultado_Laboratorio_2024.xlsx,-,-,-,-,Andamios,1,2024-04
41845,41846,-,-,2022,Feb,2022-02-08,-,Planta de beneficio,A91,A91-A,...,0a6b66ae6a7f45f0c3d39f807d609841,-,RG_L_04_Informe_Resultado_Laboratorio_2022.xlsx,-,-,-,-,Pinzas,0,2022-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71796,71797,-,-,2024,Abr,2024-04-19,-,Planta de beneficio,A91,A91-A,...,ffe237fec890d319a5893794ea244108,-,RG_L_04_Informe_Resultado_Laboratorio_2024.xlsx,-,-,-,-,Repeladora,1,2024-04
71797,71798,-,-,2022,Sep,2022-09-29,-,Planta de beneficio,A91,A91-A,...,ffeb14efbfd133aa87545859a79bd946,-,RG_L_04_Informe_Resultado_Laboratorio_2022.xlsx,-,-,-,-,Cuchillos,0,2022-09
71801,71802,-,-,2023,Ago,2023-08-25,-,Planta de beneficio,A91,A91-A,...,ffed2f7ce82cf7e01c25c6031f0b9745,-,RG_L_04_Informe_Resultado_Laboratorio_2023.xlsx,-,-,-,-,Escabiladero,0,2023-08
71803,71804,-,-,2021,Abr,2021-04-17,-,Planta de beneficio,A91,A91-A,...,fff40a55053f2713064e99b118484db9,-,RG_L_04_Informe_Resultado_Laboratorio_2021.xlsx,-,-,-,-,Termómetros,1,2021-04


In [27]:
# analisis 4, 0,0,0,1

#prev =  (0+0+0+1)/4

In [28]:
df_comp_group = (
    df_comp
    .groupby(["Fecha de muestreo.autoCalendar.Date", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Fecha de muestreo.autoCalendar.Date",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()
cols_ord = [
    'Fecha de muestreo.autoCalendar.Date',
    'Planta porcinos',
    'Producción porcinos',
    'Frío porcinos',
    'Producto terminado'
]
df_long = df_result.melt(
    id_vars="Fecha de muestreo.autoCalendar.Date",
    value_vars=df['Etapa de proceso'].unique(),
    var_name="Etapa",
    value_name="Probabilidad"
)

# Colores corporativos asignados manualmente
color_map = {
    "Planta porcinos": "#f9ee77",
    "Producción porcinos": "#1c8074",
    "Frío porcinos": "#666666",
    "Producto terminado": "#1a494c"
}

# Gráfico de líneas
fig = px.line(
    df_long,
    x="Fecha de muestreo.autoCalendar.Date",
    y="Probabilidad",
    color="Etapa",
    color_discrete_map=color_map,
    markers=True,
    title="Probabilidad de prevalencia por etapa del proceso"
)
fig.update_layout(
    xaxis_title="Día de muestreo",
    yaxis_title="Probabilidad",
    template="simple_white",
    legend_title="Etapa del proceso",
    font=dict(size=12, family="Arial")
)
# --- Franja desde junio 2025 hasta el final de los datos ---
x_start = pd.Timestamp("2024-06-20")
date_col = "Fecha de muestreo.autoCalendar.Date"
x_end   = df_long[date_col].max()  # fin real de tus datos

fig.add_annotation(
    x=x_start, y=1.02, xref="x", yref="paper",
    showarrow=False, text="Reducción Global desde junio 2024",
    font=dict(color="#1c8074", size=11)
)
# Sombreado vertical (franja)
fig.add_vrect(
    x0=x_start, x1=x_end,
    fillcolor="#666666",  # teal corporativo con transparencia
    line_width=0,
    layer="below",  # debajo de las líneas
)
fig.show()
s3.save_plotly_html(fig, "probabilidad_prevalencia_etapas.html")


In [29]:
df["Microorganismo"].unique()

array(['Rec. E. coli', 'Rec. Coliformes totales', 'Det. Salmonella spp.',
       'Det. E. coli O157:H7', 'Antiobióticos', 'Met. pesados',
       'Det. Trichinella', 'Rec. Aerobios mesófilos'], dtype=object)

In [30]:
df_comp_group = (
    df_comp
    .groupby(["Fecha de muestreo.autoCalendar.Date", "Microorganismo"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Fecha de muestreo.autoCalendar.Date",
    columns="Microorganismo",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()


df_result["Fecha de muestreo.autoCalendar.Date"] = pd.to_datetime(df_result["Fecha de muestreo.autoCalendar.Date"])

# Reordenar por fecha (por si acaso)
df_ = df_result.sort_values("Fecha de muestreo.autoCalendar.Date")


df_long = df_.melt(
    id_vars="Fecha de muestreo.autoCalendar.Date",
    value_vars=df["Microorganismo"].unique(),
    var_name="Etapa",
    value_name="Probabilidad"
)

# Colores corporativos asignados manualmente
color_map = {
    'Rec. E. coli': "#1C8074",       # teal
    'Rec. Coliformes totales': "#1A494C",   # verde oscuro
    'Det. Salmonella spp.': "#94AF92",
    'Det. E. coli O157:H7':  "#C9C9C9"
}

# Gráfico de líneas
fig = px.line(
    df_long,
    x="Fecha de muestreo.autoCalendar.Date",
    y="Probabilidad",
    color="Etapa",
    color_discrete_map=color_map,
    markers=True,
    title="Probabilidad de prevalencia por tipo de Microorganismo",
)

# Estética
fig.update_layout(
    xaxis_title="Dia de muestreo",
    yaxis_title="Probabilidad",
    template="simple_white",
    legend_title="Tipo Microorganismo",
    font=dict(size=12, family="Arial")
)
# --- Franja desde junio 2025 hasta el final de los datos ---
x_start = pd.Timestamp("2024-06-20")
date_col = "Fecha de muestreo.autoCalendar.Date"
x_end   = df_long[date_col].max()  # fin real de tus datos

fig.add_annotation(
    x=x_start, y=1.02, xref="x", yref="paper",
    showarrow=False, text="Reducción Global desde junio 2024",
    font=dict(color="#1c8074", size=11)
)
# Sombreado vertical (franja)
fig.add_vrect(
    x0=x_start, x1=x_end,
    fillcolor="#666666",  # teal corporativo con transparencia
    line_width=0,
    layer="below",  # debajo de las líneas
)
s3.save_plotly_html(fig, "probabilidad_prevalencia_microorganismo.html")
fig.show()


In [31]:

df_comp_group = (
    df[df['Fecha de muestreo.autoCalendar.Date']>='2023']
    .groupby(["year_month", "Microorganismo"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="year_month",
    columns="Microorganismo",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()


# Reordenar por fecha (por si acaso)
df_ = df_result.sort_values("year_month")
df_long = df_.melt(
    id_vars="year_month",
    value_vars=df["Microorganismo"].unique(),
    var_name="Etapa",
    value_name="Probabilidad"
)

# Colores corporativos asignados manualmente
color_map = {
    'Rec. E. coli': "#1C8074",       # teal
    'Rec. Coliformes totales': "#1A494C",   # verde oscuro
    'Det. Salmonella spp.': "#94AF92",
    'Det. E. coli O157:H7':  "#C9C9C9"
}

# Gráfico de líneas
fig = px.line(
    df_long,
    x="year_month",
    y="Probabilidad",
    color="Etapa",
    color_discrete_map=color_map,
    markers=True,
    title="Probabilidad de prevalencia por tipo de Microorganismo",
)

# Estética
fig.update_layout(
    xaxis_title="Mes de muestreo",
    yaxis_title="Probabilidad",
    template="simple_white",
    legend_title="Tipo Microorganismo",
    font=dict(size=12, family="Arial")
)
 #"--- Franja desde junio 2025 hasta el final de los datos ---
x_start = pd.Timestamp("2024-06-01")
date_col = "year_month"
x_end   = df_long[date_col].max()  # fin real de tus datos

fig.add_annotation(
    x=x_start, y=1.02, xref="x", yref="paper",
    showarrow=False, text="Reducción Global desde junio 2024",
    font=dict(color="#1c8074", size=11)
)
# Sombreado vertical (franja)
fig.add_vrect(
    x0=x_start, x1=x_end,
    fillcolor="#666666",  # teal corporativo con transparencia
    line_width=0,
    layer="below",  # debajo de las líneas
)
s3.save_plotly_html(fig, "probabilidad_prevalencia_microorganismo_mes.html")    


fig.show()


In [32]:

df_comp_group = (
    df_comp
    .groupby(["Muestra", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)
df_result = df_comp_group.pivot(
    index="Muestra",
    columns="Etapa de proceso",
    values="prev"
)
df_result = df_result.sort_index(axis=1).reset_index()
df_result

Etapa de proceso,Muestra,Frío porcinos,Planta porcinos,Producción porcinos,Producto terminado
0,Ambiente área de canastas,NaN,NaN,1.000000,NaN
1,Andamio,NaN,NaN,0.500000,NaN
2,Andamio (Pnc O Tol 0),NaN,0.250000,NaN,NaN
3,Andamio Muelle,0.302326,NaN,NaN,NaN
4,Astilladora,NaN,0.458955,NaN,NaN
...,...,...,...,...,...
72,Termómetro Auxiliar Calidad,0.174603,NaN,NaN,NaN
73,Termómetro Cava,0.305556,NaN,NaN,NaN
74,Termómetro Punzón Cava,0.267045,NaN,NaN,NaN
75,Toombler,NaN,NaN,0.342593,NaN


In [33]:
#TODO: ¿Por que se llama tratamiento?
# TODO, en porcinos existen label de bovinos...mal tabulado

df_comp_group = (
    df
    .groupby(["Tratamiento", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Tratamiento",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()
df_result

Etapa de proceso,Tratamiento,Frío porcinos,Planta porcinos,Producción porcinos,Producto terminado
0,-,0.500000,0.000000,0.411643,0.413153
1,Oper Bov,NaN,0.125000,NaN,NaN
2,Oper Bovino,0.500000,0.435714,NaN,NaN
3,Oper Por,0.062500,0.117647,0.156250,NaN
4,Oper Porcino,0.399038,0.463722,0.434564,NaN
5,Preoper Bov,NaN,0.071429,NaN,NaN
6,Preoper Bovino,0.437500,0.225806,NaN,NaN
7,Preoper Por,0.023810,0.028571,0.107143,NaN
8,Preoper Porcino,0.293224,0.319264,0.280083,NaN


In [34]:
df_comp["Etapa de proceso"].unique()

array(['Planta porcinos', 'Producto terminado', 'Frío porcinos',
       'Producción porcinos'], dtype=object)

In [35]:
df["Etapa de proceso"].unique()

array(['Producto terminado', 'Planta porcinos', 'Producción porcinos',
       'Frío porcinos'], dtype=object)

In [36]:
df_comp_group = (
    df
    .groupby(["Microorganismo", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Microorganismo",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()


micro_col = "Microorganismo"
stage_cols = ['Producción porcinos', 'Frío porcinos', 'Producto terminado']  # orden fijo

# Normaliza si vienen en %
vals = df_result[stage_cols].to_numpy().ravel()
finite = vals[np.isfinite(vals)]
if finite.size and np.nanmax(finite) > 1.5:
    df_result[stage_cols] = df_result[stage_cols] / 100.0

# Matriz y etiquetas
mat = df_result.set_index(micro_col)[stage_cols]
Z = mat.to_numpy()
x_labels = stage_cols
y_labels = mat.index.tolist()

# ----------------------------
# Paleta corporativa (0 → 1)
# ----------------------------
colorscale = [
    (0.00, "#E6ECD8"),  # claro
    (0.33, "#94AF92"),
    (0.66, "#1C8074"),
    (1.00, "#1A494C"),  # oscuro
]

# ----------------------------
# Gráfico Plotly
# ----------------------------
fig = px.imshow(
    Z,
    x=x_labels,
    y=y_labels,
    color_continuous_scale=colorscale,
    zmin=0, zmax=1,               # escala fija 0–1
    text_auto=".0%",              # anotar valores en %
    aspect="auto"
)

# Ejes y estilo
fig.update_xaxes(title_text="Etapa de proceso",
                 categoryorder="array", categoryarray=x_labels)
fig.update_yaxes(title_text="Microorganismo",
                 categoryorder="array", categoryarray=y_labels)

fig.update_layout(
    title=dict(text="Mapa de calor de probabilidad por etapa y microorganismo",
               font=dict(color="#1C8074", size=18), x=0.5),
    template="simple_white",
    coloraxis_colorbar=dict(title="Probabilidad", tickformat=".0%"),
    font=dict(size=12, family="Arial"),
)

# (Opcional) separadores sutiles entre celdas
fig.update_traces(xgap=1, ygap=1)
s3.save_plotly_html(
    fig,
   "heatmap_microorganismo_etapa_plotly.html"
)
fig.show()

In [37]:
df_2024 = df[df["Fecha de muestreo.autoCalendar.Date"]>='2025'].copy()
df_2024 = df_2024.sort_values(["year_month"])
for ym in df_2024["year_month"].unique():

    df_2024_ = df_2024[df_2024["year_month"] == ym]
    df_comp_group = (
        df_2024_
        .groupby(["Microorganismo", "Etapa de proceso"])
        .agg(prev=("deteccion", "mean"))
        .reset_index()
    )

    df_result = df_comp_group.pivot(
        index="Microorganismo",
        columns="Etapa de proceso",
        values="prev"
    )

    df_result = df_result.sort_index(axis=1).reset_index()


    micro_col = "Microorganismo"
    stage_cols = ['Producción porcinos','Frío porcinos', 'Producto terminado']  # orden fijo

    # Normaliza si vienen en %
    vals = df_result[stage_cols].to_numpy().ravel()
    finite = vals[np.isfinite(vals)]
    if finite.size and np.nanmax(finite) > 1.5:
        df_result[stage_cols] = df_result[stage_cols] / 100.0

    # Matriz y etiquetas
    mat = df_result.set_index(micro_col)[stage_cols]
    Z = mat.to_numpy()
    x_labels = stage_cols
    y_labels = mat.index.tolist()

    # ----------------------------
    # Paleta corporativa (0 → 1)
    # ----------------------------
    colorscale = [
        (0.00, "#E6ECD8"),  # claro
        (0.33, "#94AF92"),
        (0.66, "#1C8074"),
        (1.00, "#1A494C"),  # oscuro
    ]

    # ----------------------------
    # Gráfico Plotly
    # ----------------------------
    fig = px.imshow(
        Z,
        x=x_labels,
        y=y_labels,
        color_continuous_scale=colorscale,
        zmin=0, zmax=1,               # escala fija 0–1
        text_auto=".0%",              # anotar valores en %
        aspect="auto"
    )

    # Ejes y estilo
    fig.update_xaxes(title_text="Etapa de proceso",
                     categoryorder="array", categoryarray=x_labels)
    fig.update_yaxes(title_text="Microorganismo",
                     categoryorder="array", categoryarray=y_labels)

    fig.update_layout(
        title=dict(text=f"Mapa de calor de probabilidad por etapa y microorganismo {ym}",
                   font=dict(color="#1C8074", size=18), x=0.5),
        template="simple_white",
        coloraxis_colorbar=dict(title="Probabilidad", tickformat=".0%"),
        font=dict(size=12, family="Arial"),
    )

    # (Opcional) separadores sutiles entre celdas
    fig.update_traces(xgap=1, ygap=1)
    fig.show()
    s3.save_plotly_html(fig, f"heatmap_microorganismo_etapa_plotly_{ym}.html")

# Guardar a HTML
#
#fig.show()

In [38]:
df_comp_group = (
    df
    .groupby(["Microorganismo", "Categoría"])
    .agg(prev=("deteccion", "mean"),cantidad=("deteccion","count"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Microorganismo",
    columns="Categoría",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()


micro_col = "Microorganismo"
stage_cols = df["Categoría"].unique()# orden fijo

# Normaliza si vienen en %
vals = df_result[stage_cols].to_numpy().ravel()
finite = vals[np.isfinite(vals)]
if finite.size and np.nanmax(finite) > 1.5:
    df_result[stage_cols] = df_result[stage_cols] / 100.0

# Matriz y etiquetas
mat = df_result.set_index(micro_col)[stage_cols]
Z = mat.to_numpy()
x_labels = stage_cols
y_labels = mat.index.tolist()

# ----------------------------
# Paleta corporativa (0 → 1)
# ----------------------------
colorscale = [
    (0.00, "#E6ECD8"),  # claro
    (0.33, "#94AF92"),
    (0.66, "#1C8074"),
    (1.00, "#1A494C"),  # oscuro
]

# ----------------------------
# Gráfico Plotly
# ----------------------------
fig = px.imshow(
    Z,
    x=x_labels,
    y=y_labels,
    color_continuous_scale=colorscale,
    zmin=0, zmax=1,               # escala fija 0–1
    text_auto=".0%",              # anotar valores en %
    aspect="auto"
)

# Ejes y estilo
fig.update_xaxes(title_text="Categoría",
                 categoryorder="array", categoryarray=x_labels)
fig.update_yaxes(title_text="Microorganismo",
                 categoryorder="array", categoryarray=y_labels)

fig.update_layout(
    title=dict(text="Mapa de calor de probabilidad por categoria y microorganismo",
               font=dict(color="#1C8074", size=18), x=0.5),
    template="simple_white",
    coloraxis_colorbar=dict(title="Probabilidad", tickformat=".0%"),
    font=dict(size=12, family="Arial"),
)

# (Opcional) separadores sutiles entre celdas
fig.update_traces(xgap=1, ygap=1)

# Mostrar
fig.show()


s3.save_plotly_html(fig, "heatmap_microorganismo_categoria_plotly.html")

In [39]:

for ym in df_2024["year_month"].unique():

    df_2024_ = df_2024[df_2024["year_month"] == ym]

    df_comp_group = (
        df_2024_
        .groupby(["Microorganismo", "Categoría"])
        .agg(prev=("deteccion", "mean"),cantidad=("deteccion","count"))
        .reset_index()
    )

    df_result = df_comp_group.pivot(
        index="Microorganismo",
        columns="Categoría",
        values="prev"
    )

    df_result = df_result.sort_index(axis=1).reset_index()


    micro_col = "Microorganismo"
    stage_cols = df_2024_["Categoría"].unique()# orden fijo

    # Normaliza si vienen en %
    vals = df_result[stage_cols].to_numpy().ravel()
    finite = vals[np.isfinite(vals)]
    if finite.size and np.nanmax(finite) > 1.5:
        df_result[stage_cols] = df_result[stage_cols] / 100.0

    # Matriz y etiquetas
    mat = df_result.set_index(micro_col)[stage_cols]
    Z = mat.to_numpy()
    x_labels = stage_cols
    y_labels = mat.index.tolist()

    # ----------------------------
    # Paleta corporativa (0 → 1)
    # ----------------------------
    colorscale = [
        (0.00, "#E6ECD8"),  # claro
        (0.33, "#94AF92"),
        (0.66, "#1C8074"),
        (1.00, "#1A494C"),  # oscuro
    ]

    # ----------------------------
    # Gráfico Plotly
    # ----------------------------
    fig = px.imshow(
        Z,
        x=x_labels,
        y=y_labels,
        color_continuous_scale=colorscale,
        zmin=0, zmax=1,               # escala fija 0–1
        text_auto=".0%",              # anotar valores en %
        aspect="auto"
    )

    # Ejes y estilo
    fig.update_xaxes(title_text="Categoría",
                     categoryorder="array", categoryarray=x_labels)
    fig.update_yaxes(title_text="Microorganismo",
                     categoryorder="array", categoryarray=y_labels)

    fig.update_layout(
        title=dict(text="Mapa de calor de probabilidad por categoria y microorganismo",
                   font=dict(color="#1C8074", size=18), x=0.5),
        template="simple_white",
        coloraxis_colorbar=dict(title="Probabilidad", tickformat=".0%"),
        font=dict(size=12, family="Arial"),
    )

    # (Opcional) separadores sutiles entre celdas
    fig.update_traces(xgap=1, ygap=1)

    # Mostrar
    # fig.show()

    # Guardar a HTML
    fig.show()
    s3.save_plotly_html(fig, f"heatmap_microorganismo_categoria_plotly_{ym}.html")

In [40]:
df_comp_group = (
    df_comp
    .groupby(["Microorganismo", "Categoría"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Microorganismo",
    columns="Categoría",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result


Categoría,Microorganismo,Agua,Ambiente,Manipulador,No aplica,POES
0,Antiobióticos,NaN,NaN,NaN,0.100000,NaN
1,Det. E. coli O157:H7,NaN,NaN,NaN,0.019608,NaN
2,Det. Salmonella spp.,NaN,NaN,NaN,0.051643,NaN
3,Det. Trichinella,NaN,NaN,NaN,0.000000,NaN
4,Met. pesados,NaN,NaN,NaN,0.000000,NaN
5,Rec. Aerobios mesófilos,NaN,1.0,NaN,NaN,NaN
6,Rec. Coliformes totales,0.0,NaN,0.166667,0.592213,0.542567
7,Rec. E. coli,0.0,NaN,0.166667,0.516340,0.149440


In [42]:
df_comp_group = (
    df_comp
    .groupby(["Inductor", "Categoría"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Inductor",
    columns="Categoría",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result


Categoría,Inductor,Agua,Ambiente,Manipulador,No aplica,POES
0,Control de proceso,0.0,1.0,0.166667,NaN,0.346004
1,Producto terminado,NaN,NaN,NaN,0.419305,NaN


In [43]:
df_comp_group = (
    df_comp
    .groupby(["Patogenicidad", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Patogenicidad",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result


Etapa de proceso,Patogenicidad,Frío porcinos,Planta porcinos,Producción porcinos,Producto terminado
0,Indicador,0.319418,0.356043,0.343987,0.553102
1,Patógeno,NaN,NaN,NaN,0.049419


In [44]:
df_comp_group = (
    df_comp
    .groupby(["Especie/origen", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Especie/origen",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result


Etapa de proceso,Especie/origen,Frío porcinos,Planta porcinos,Producción porcinos,Producto terminado
0,No aplica,0.319418,0.356043,0.343987,NaN
1,Porcinos,NaN,NaN,NaN,0.419305


In [45]:
df_comp_group = (
    df_comp
    .groupby(["Manipulador", "Etapa de proceso"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Manipulador",
    columns="Etapa de proceso",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result


Etapa de proceso,Manipulador,Frío porcinos,Planta porcinos,Producción porcinos,Producto terminado
0,-,0.319418,0.356043,0.347639,0.419305
1,Camilo A Munera,NaN,NaN,0.000000,NaN
2,Edilson Torres,NaN,NaN,0.000000,NaN
3,Edward Mosquera,NaN,NaN,0.000000,NaN
4,Freider Arroyo,NaN,NaN,1.000000,NaN
5,Javier Mosa,NaN,NaN,0.000000,NaN
6,Jeferson Tabares,NaN,NaN,0.000000,NaN
7,Jhon De La Cruz,NaN,NaN,0.000000,NaN
8,Jose Gonzalez,NaN,NaN,0.000000,NaN
9,Juan Carlos Gonzalez,NaN,NaN,1.000000,NaN


In [46]:
df_comp_group = (
    df
    .groupby(["Manipulador", "Fecha de muestreo.autoCalendar.Date"])
    .agg(prev=("deteccion", "mean"))
    .reset_index()
)

df_result = df_comp_group.pivot(
    index="Fecha de muestreo.autoCalendar.Date",
    columns="Manipulador",
    values="prev"
)

df_result = df_result.sort_index(axis=1).reset_index()

df_result

Manipulador,Fecha de muestreo.autoCalendar.Date,-,Adalberto Ruiz,Adrian Perez,Agustin Rodriguez,Albeiro David,Albin Mosquera,Aldubar Casanova,Aldwar Casanova,Alejandro Bohorquez,...,Yefferson Mosquera,Yefren Hernandez,Yeider Mosquera,Yeiler Mosquera Teran,Yeison Arrieta,Yeison Cardona,Yeison Ochoa,Yeison Santos,Yeraldin Bedoya,Yeraldine Bedoya
0,2018-01-26,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-24,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-12-10,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-02,0.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,2025-08-20,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1840,2025-08-21,0.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1841,2025-08-22,0.071429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1842,2025-08-23,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
